In [40]:
import urllib.request
from bs4 import BeautifulSoup

companies = ['facebook','alibaba']
for company in companies:
    quote_page = "https://news.google.com/news/search/section/q/$REPLACE/$REPLACE?hl=en&gl=US&ned=us"
    quote_page = quote_page.replace("$REPLACE", company)
    print(company)
    page = urllib.request.urlopen(quote_page)
    soup = BeautifulSoup(page, "html.parser")

    #look for aria-level="2"
    arias = soup.find_all('a')
    links = []
    for aria in arias:
        if aria.get('href')[0:1]=="h":
            links.append(aria.get('href'))

    links = list(dict.fromkeys(links))
    for link in links[22:30]:
        print(link)

facebook
https://www.theverge.com/2018/2/13/17009114/facebook-campbell-brown-code-media-news-publishers
https://www.recode.net/2018/2/12/17005058/facebook-help-publishers-news-feed-algorithm-campbell-brown-adam-mosseri-code-media
https://www.fastcompany.com/40530310/apple-and-facebook-work-out-news-subscription-deal
https://mashable.com/2018/02/12/facebook-slogan-good-for-business-leave-code-media/
https://techcrunch.com/2018/02/13/facebook-pushes-for-more-personal-updates-with-launch-of-new-lists-feature/
https://www.engadget.com/2018/02/13/facebooks-lists-encourage-personal-oversharing/
https://thenextweb.com/facebook/2018/02/13/facebooks-colorful-new-feature-makes-public-lists-thing/
http://fortune.com/2018/02/13/facebook-fake-news-wired-article/
alibaba
https://www.cnbc.com/2018/02/12/concerns-over-alibaba-led-digital-free-trade-zone-in-malaysia.html
https://www.malaysiakini.com/news/412130
https://www.fool.com/investing/2018/02/13/the-5-most-important-figures-from-alibabas-q3-repo